In [1]:
import pandas as pd
import numpy as np
import datetime 
import os 

In [2]:
#read in the latest dataset
time_series = pd.read_csv("../../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
#time_series.head()

In [3]:
#keep only rows with NC county data
time_series = time_series[time_series["Province_State"] == "North Carolina"]
time_series = time_series[time_series["Admin2"] != "Out of NC"]
time_series = time_series[time_series["Admin2"] != "Unassigned"]
#time_series.head()

In [4]:
#define a dataframe with just the county information
nc_counties = time_series[["FIPS", "Admin2","Province_State","Lat","Long_"]]
nc_counties.head()

,FIPS,Admin2,Province_State,Lat,Long_
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570
1898,37007.0,Anson,North Carolina,34.974032,-80.099533
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627


In [5]:
#define a variable to define the end date of data to be included
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

128

In [6]:
#define a dataframe with just the total cases by county each day
dates = time_series.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20
1895,0,0,0,0,0,0,0,0,0,0,...,149,152,173,183,183,190,196,196,203,217
1896,0,0,0,0,0,0,0,0,0,0,...,8,10,12,12,13,14,15,15,18,21
1897,0,0,0,0,0,0,0,0,0,0,...,7,7,8,8,8,10,10,10,11,11
1898,0,0,0,0,0,0,0,0,0,0,...,37,38,38,38,39,40,41,43,46,49
1899,0,0,0,0,0,0,0,0,0,0,...,7,8,13,13,14,16,17,17,26,27


In [7]:
#define an empty dataframe to transpose the cases by date into a new row for each county-date pair
nc_time_series = pd.DataFrame(columns = ["FIPS","Admin2","Province_State","Lat","Long_","Confirmed","Date"])

In [8]:
#fill in the dataframe
cols = dates.columns

for i in range(0,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Date"] = cols[i]
    county_time = nc_counties.join(one_day)
    nc_time_series = nc_time_series.append(county_time)
#    county_time.append(one_day)
#    print(one_day)
nc_time_series.head()


,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Date
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761,0,3/4/20
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519,0,3/4/20
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570,0,3/4/20
1898,37007.0,Anson,North Carolina,34.974032,-80.099533,0,3/4/20
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627,0,3/4/20


In [9]:
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)

In [10]:
nc_time_series["Date"] = pd.to_datetime(nc_time_series.Date)
#nc_time_series.dtypes

In [11]:
#convert Date column to format that will be uniform across use cases
#nc_time_series["Date"] = nc_time_series["Date"].dt.to_period("D")

In [12]:
#add a column with a unique value that can be used to establish a relationship with other datasheets
nc_time_series["Unique"] = nc_time_series["Admin2"]+", "+nc_time_series["Date"].astype(str)

#add a column so location will be readable by power BI
nc_time_series["Location"] = nc_time_series["Admin2"]+" County "+nc_time_series["Province_State"]

In [13]:
#nc_time_series.tail()

In [14]:
#nc_time_series.head()

In [15]:
nc_time_series.sort_values(by = ["Unique","Date"], inplace = True, ignore_index = True)
#nc_time_series.head()

In [16]:
nc_time_series["New_Cases"] = nc_time_series["Confirmed"]-nc_time_series["Confirmed"].shift(1)
#nc_time_series.tail()

In [17]:
nc_time_series["Rolling_Avg"] = (nc_time_series["Confirmed"]+nc_time_series["Confirmed"].shift(1)+nc_time_series["Confirmed"].shift(2)+nc_time_series["Confirmed"].shift(3)+nc_time_series["Confirmed"].shift(4)+nc_time_series["Confirmed"].shift(5)+nc_time_series["Confirmed"].shift(6)+nc_time_series["Confirmed"].shift(7)+nc_time_series["Confirmed"].shift(8)+nc_time_series["Confirmed"].shift(9)+nc_time_series["Confirmed"].shift(10)+nc_time_series["Confirmed"].shift(11)+nc_time_series["Confirmed"].shift(12)+nc_time_series["Confirmed"].shift(13))/14
#nc_time_series.tail()

In [18]:
nc_time_series["New_Case_Roll_Avg"] = (nc_time_series["New_Cases"]+nc_time_series["New_Cases"].shift(1)+nc_time_series["New_Cases"].shift(2)+nc_time_series["New_Cases"].shift(3)+nc_time_series["New_Cases"].shift(4)+nc_time_series["New_Cases"].shift(5)+nc_time_series["New_Cases"].shift(6)+nc_time_series["New_Cases"].shift(7)+nc_time_series["New_Cases"].shift(8)+nc_time_series["New_Cases"].shift(9)+nc_time_series["New_Cases"].shift(10)+nc_time_series["New_Cases"].shift(11)+nc_time_series["New_Cases"].shift(12)+nc_time_series["New_Cases"].shift(13))/14
#nc_time_series.head()

In [19]:
timeframe = 14
nc_time_series["New_Case_Change"] = (nc_time_series["New_Case_Roll_Avg"]-nc_time_series["New_Case_Roll_Avg"].shift(14))/timeframe
#nc_time_series.tail()

In [20]:
nc_time_series["Normalized_Change"] = nc_time_series.apply(lambda row: row.New_Case_Change/row.New_Case_Roll_Avg if row.New_Case_Roll_Avg !=0 else 0, axis = 1)
#nc_time_series.tail()

In [21]:
offset2 = datetime.timedelta(days = 14)
firstdate = startdate + offset2 
firstdate

datetime.datetime(2020, 3, 18, 0, 0)

In [22]:
nc_time_series = nc_time_series[nc_time_series["Date"]>firstdate]
#nc_time_series.head()

In [23]:
offset3 = datetime.timedelta(days = 2)
olddate = today - offset3
olddate

datetime.datetime(2020, 5, 16, 23, 15, 7, 621409)

In [24]:
nc_time_series_today = nc_time_series[nc_time_series["Date"] > olddate]

In [25]:
nc_time_series = nc_time_series[["Admin2","Province_State","Location","Unique","Date","Lat","Long_","Confirmed","Rolling_Avg",
                                "New_Cases","New_Case_Roll_Avg","New_Case_Change","Normalized_Change"]]
nc_time_series_today = nc_time_series_today[["Admin2","Province_State","Location","Unique","Date","Lat","Long_","Confirmed","Rolling_Avg",
                                "New_Cases","New_Case_Roll_Avg","New_Case_Change","Normalized_Change"]]
#nc_time_series.head()

In [26]:
nc_time_series.to_excel("../clean_data/nc_time_series.xlsx")
nc_time_series_today.to_excel("../clean_data/nc_time_series_today.xlsx")